# Извлечение ключевого фрагмента из текста с помощью NER

В этом ноутбуке я дообучаю модель для выделения ключевых фрагментов из текста. Ключевые фрагменты можно рассматривать как именованные сущности, тогда задача будет называться NER. 
И сведется к классификации отдельных слов.

Данный пайплайн в целом подходит для любой задачи на выделение сущностей в тексте.

In [1]:
!pip install -qr ./requirements.txt

In [1]:
import os
import re
import json
import random
from pathlib import Path

from tqdm.notebook import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
# чтобы удобно подгружать nlp модели из большого открытого хаба
# и удобно предобработывать тексты, взял transformers от HuggingFace
from transformers import AutoTokenizer, AutoModel
# Для оберток над train loop и датасетов взял pytorch_lightning, тк был знаком с ним ранее
# хотя было бы логичнее по возможности испльзовать альтернативы из transformers
import pytorch_lightning as pl
import torch

In [2]:
# во время pytorch_lightning training loop вылезает предупреждение
# которое просит отключить эту штуку
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

Для хранинения и параметов эксперимента написал свой класс конфига, чтобы удобно хранить их в yaml  
и парсить в объект с типизированными полями

In [3]:
# при изменении структуры конфига надо переимпортировать его
import importlib
import helpers.config as config
importlib.reload(config)

<module 'helpers.config' from '/home/gk/projects/kontur/solution/helpers/config.py'>

Считаем конфиг и зададим сид

In [4]:
CONFIG = config.Config.from_yaml_file(Path("./train_config.yml"))

В качестве модели для начала я взял предобученный на русской Википедии [BERT (rubert-base-cased) от DeepPavlov](https://huggingface.co/DeepPavlov/rubert-base-cased) на 180M параметров.
Он показал себя хорошо, поэтому я не стал тратить время тест других.
Также можно было бы еще поэкспериментировать с другими версиями BERT из HuggingFace хаба.
Например с [rubert-tiny](https://huggingface.co/cointegrated/rubert-tiny) или [rubert-tiny2-finetuned-ner](https://huggingface.co/Evolett/rubert-tiny2-finetuned-ner).
Хотя из-за их размера точность скорее всего была бы хуже.

In [5]:
CONFIG

{
    "data": {
        "train_data_path": "./data/train.json",
        "test_data_path": "./data/test.json",
        "split": [
            0.7,
            0.2,
            0.1
        ]
    },
    "model": {
        "model_name": "DeepPavlov/rubert-base-cased",
        "unfreeze_layers": [
            "encoder.layer.10",
            "encoder.layer.11"
        ]
    },
    "training": {
        "project_name": "kontur-text-extraction",
        "experiment_name": "exp6-final-test",
        "description": "training to be displaed in notebook",
        "batch_size": 32,
        "epochs": 30,
        "precision": 16,
        "seed": 42
    }
}

Зафиксируем рандом для воспроизводимости

In [6]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
pl.seed_everything(CONFIG.training.seed, workers=True)
torch.set_float32_matmul_precision('medium')

Global seed set to 42


## Предобработка данных

Для аннотации был использован немного измененный [BIO](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)) формат. Тк в нашей задаче сущности относительно длинные, а итоговая метрика accuracy очень чувствительна даже к одному лишнему символу, я добавил дополнительный тэг окончания **E-{name}**, посчитав, что так модель будет уделять больше внимания предсказанию конца фразы. Также с тэгом E будет удобнее доставать предсказание из текста, тк мы просто сможем выбрать самый вероятный токен для начала и окончания фразы.  
  
Короткие названия для запросов, которые я далее буду использовать:  
* EXEC  - обеспечение исполнения контракта
* GUAR - обеспечение гарантийных обязательств

In [7]:
tags = ['O', 'B-EXEC', 'B-GUAR', 'I-EXEC', 'I-GUAR', 'E-EXEC', 'E-GUAR']

In [8]:
# загрузим токенайзер из HuggingFace хаба, соответствующий модели, которуя я выбрал
tokenizer = AutoTokenizer.from_pretrained(CONFIG.model.model_name)

Заранее токенизируем запросы и посчитаем их длину в токенах

In [9]:
labels_tokenized = {
    "EXEC": {
        "data": tokenizer(
            "обеспечение исполнения контракта", # вот строка запроса
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_offsets_mapping=True,
            return_tensors='pt'
        )
    },
    "GUAR": {
        "data": tokenizer(
        "обеспечение гарантийных обязательств", # и вот
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_offsets_mapping=True,
            return_tensors='pt'
        )
    },
}

labels_tokenized["EXEC"]["len"] = len(labels_tokenized['EXEC']["data"]['input_ids'][0])
labels_tokenized["GUAR"]["len"] = len(labels_tokenized['GUAR']["data"]['input_ids'][0])

Считаем трейн данные

In [10]:
with open(CONFIG.data.train_data_path) as f:
    data = json.load(f)

Разделим данные на 4 выборки: трейн, вал, и две тестовые  
Первая тестовая выборка нужна чтобы проверить качество модели именно на предобработанных данных.  
Вторая - на сырых  
Если бы я тестировал модель только на одной выборке, то при плохих результах я бы не мог с уверенностью сказать,  
это модель плохо обучились или мой подход к инференсу работает плохо. 

In [11]:
val_size = CONFIG.data.split[1] / sum(CONFIG.data.split[1:])

# детерминированно перемешиваем
temp_data, final_test_data = train_test_split(data, test_size=CONFIG.data.split[2], shuffle=True, random_state=CONFIG.training.seed)
train_data, temp_data = train_test_split(temp_data, train_size=CONFIG.data.split[0], shuffle=False)
val_data, test_data = train_test_split(temp_data, train_size=val_size, shuffle=False)
print(f"Размер итоговой тестовой выборки: {len(final_test_data)}")
print(f"Размер обучающей выборки: {len(train_data)}")
print(f"Размер валидационной выборки: {len(val_data)}")
print(f"Размер тестовой выборки: {len(test_data)}")

Размер итоговой тестовой выборки: 180
Размер обучающей выборки: 1133
Размер валидационной выборки: 324
Размер тестовой выборки: 162


Здесь я формирую датасет и каждому токену из предложения сопоставляю тэг.  
Тк большинство предоставленных текстов содержат больше чем 512 токенов, их придется поделить на куски поменьше,  
чтобы BERT мог с ними работать.  
Для этого беру рандомные срезы вокруг фразы которую нужно вытащить, длины не более 400 (на всякий случай).  
Срезы беру вокруг фраз, чтобы не обрезать их (чтобы в сэмпле всегда было конец и начало).  
Мне кажется, что модели будет сложнее будет правильно выделить фразу в обрезанном тексте, особенно, если было обрезано начало искомой фразы. На инференсе я учту, что модель училась на "хороших" сэмплах  
Потом добавляю к каждому получившемуся сэмплу токены из запроса.  
Получится типо того:  
"обеспечение исполнения контракта [SEP] ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запроса котир..."  
  
Тк нам нужно искать сущности разного типа в разных случаях, и мы хотим делать это одной моделью, мы как бы сообщаем модели, что вообще она должна найти и она учитывает это при предсказании токена

In [12]:
from helpers.utils import generate_positive_slices, generate_negative_slices, pad_to_length, find_value_in_list


def preprocess_data(raw_data):
    tokenized_data = []
    for item in tqdm(raw_data):
        text = item['text']
        label = item['label']
        extracted_part = item['extracted_part']
        answer_start_symbol_idx = extracted_part['answer_start'][0]
        answer_end_symbol_idx = extracted_part['answer_end'][0]
        
        # для большей понятность заменяю 0 на -1
        if answer_start_symbol_idx == 0 and answer_end_symbol_idx == 0:
            answer_start_symbol_idx, answer_end_symbol_idx = -1, -1

        if label == 'обеспечение исполнения контракта':
            short_label = 'EXEC'
        else:
            short_label = 'GUAR'

        encoding = tokenizer(
            text,
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_offsets_mapping=True,
            return_tensors='pt'
        )

        token_tag_ids = []
        # беру индекс начала и конца каждого токена и смотрю где он находится
        for i, token_offset in enumerate(encoding['offset_mapping'][0]):
            token_start, token_end = token_offset[0], token_offset[1]

            # если индекс символа начала фразы внутри токена, то токен является началом (B)
            if token_start <= answer_start_symbol_idx <= token_end:
                token_tag_ids.append(tags.index(f"B-{short_label}"))
            
            # если индекс символа конца фразы внутри токена, то токен является концом (E)
            elif token_start <= answer_end_symbol_idx <= token_end:
                token_tag_ids.append(tags.index(f"E-{short_label}"))

            # если токен находися между началом и концом, то он внутренний (I)
            elif answer_start_symbol_idx <= token_start and token_end <= answer_end_symbol_idx:
                token_tag_ids.append(tags.index(f"I-{short_label}"))

            # иначе токен внешний и не принадлежит фразе
            else:
                token_tag_ids.append(tags.index("O"))
        

        sentence_len = len(token_tag_ids)

        # ищу индекс конца и начала фразы в списле с тегами токенов
        start_idx = find_value_in_list(token_tag_ids, tags.index(f"B-{short_label}"))
        end_idx = find_value_in_list(token_tag_ids, tags.index(f"E-{short_label}"))

        # генерирую индексы срезов содержащих фразу и без искомой фразы
        positive_slices = generate_positive_slices(start_idx, end_idx, n=5, max_length=400)
        negative_slices = generate_negative_slices(start_idx, end_idx, sentence_len, 2, 200, 400)

        final_seq_len = 512 # в итоге хотим получить тексты длинной 512 токенов
        for start, end in positive_slices + negative_slices:
            # собираю вместе инфромацию из токенайзера для каждого среза
            data = [
                pad_to_length(torch.cat(( # делаю паддинг тензоров до длины final_seq_len
                    labels_tokenized[short_label]["data"]['input_ids'][0], # добавляю токены из запроса
                    encoding['input_ids'][0][start:end], # беру токены с номерами от start до end из сгенерированнх срезов
                )),final_seq_len, 0),
                pad_to_length(torch.cat((
                    labels_tokenized[short_label]["data"]['token_type_ids'][0],
                    encoding['token_type_ids'][0][start:end],
                )),final_seq_len, 0),
                pad_to_length(torch.cat((
                    labels_tokenized[short_label]["data"]['attention_mask'][0],
                    encoding['attention_mask'][0][start:end],
                )),final_seq_len, 0),
                pad_to_length(torch.cat((
                    # токены запроса обозначаю как внешние, можно было бы попробовать выделить из в отдельный класс Q
                    torch.tensor([tags.index("O")] * labels_tokenized[short_label]["len"]),
                    torch.tensor(token_tag_ids[start:end],)
                )), final_seq_len, tags.index("O"))
            ]

            tokenized_data.append(data)
    return tokenized_data

In [13]:
tokenized_train_data = preprocess_data(train_data)
tokenized_val_data = preprocess_data(val_data)
tokenized_test_data = preprocess_data(test_data)
print("Размеры выборок, после препроцессинга")
print(f"Размер обучающей выборки: {len(tokenized_train_data)}")
print(f"Размер валидационной выборки: {len(tokenized_val_data)}")
print(f"Размер тестовой выборки: {len(tokenized_test_data)}")

  0%|          | 0/1133 [00:00<?, ?it/s]

  0%|          | 0/324 [00:00<?, ?it/s]

  0%|          | 0/162 [00:00<?, ?it/s]

Размеры выборок, после препроцессинга
Размер обучающей выборки: 6919
Размер валидационной выборки: 1966
Размер тестовой выборки: 1020


Посмотрим как сгенерировался датасет

In [22]:
idx = 2
tokens = tokenizer.convert_ids_to_tokens(tokenized_train_data[idx][0])
token_tags = [tags[i] for i in tokenized_train_data[idx][3]]
df = pd.DataFrame({'Tokens': tokens[200:221], 'Tags': token_tags[200:221]})

In [23]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
Tokens,.,Обеспеч,##ение,исполнения,Договора,устанавливается,в,размере,5,процент,...,ов,",",а,),Цены,Договора,.,9,.,2
Tags,O,B-EXEC,I-EXEC,I-EXEC,I-EXEC,I-EXEC,I-EXEC,I-EXEC,I-EXEC,I-EXEC,...,I-EXEC,I-EXEC,I-EXEC,I-EXEC,I-EXEC,I-EXEC,E-EXEC,O,O,O


In [24]:
from torch.utils.data import DataLoader, Dataset


class TextExtractionDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    

class TextExtractionDataModule(pl.LightningDataModule):
    def __init__(self, batch_size: int = 16):
        super().__init__()
        self.batch_size = batch_size
        self.num_workers = 4

    def setup(self, stage=None):
        self.train_data = TextExtractionDataset(tokenized_train_data)
        self.val_data = TextExtractionDataset(tokenized_val_data)
        self.test_data = TextExtractionDataset(tokenized_test_data)

    def train_dataloader(self):
        return DataLoader(self.train_data, batch_size=self.batch_size, shuffle=True, num_workers=self.num_workers)

    def val_dataloader(self):
        return DataLoader(self.val_data, batch_size=self.batch_size, num_workers=self.num_workers)

    def test_dataloader(self):
        return DataLoader(self.test_data, batch_size=self.batch_size, num_workers=self.num_workers)


## Модель

Как обертку опять использовал lightning

В модель добавляем линейный слой для классификации каждого токена на 7 классов.  
Часть весов модели замораживается для уменьшения количества обучаемых параметров.  
В качестве лосса взята CE с весами, как база для классификации  
Оптимизатором выбран Adam, тк тоже база  
Scheduler взял LinearLR, тк с ним завелось нормально

In [25]:
import torch
import torch.nn as nn
from torch.optim import Adam


class BertForTextExtraction(pl.LightningModule):
    def __init__(self, model_name=None, lr=0.01):
        super().__init__()

        # вынесл lr чтобы его можно было подобрать с помощью auto_lr_find
        self.lr = lr
        # подгрузитм предобученный берт
        self.bert = AutoModel.from_pretrained(model_name)
        # добавим линейный слой для классификации каждого токена на 7 классов
        self.linear = nn.Linear(self.bert.config.hidden_size, len(tags))

        # заморозим часть вестов, чтобы не обучать модель полностью
        self.freeze_bert_layers()
        
        # в качестве лосса берем базовый CE
        # веса для классов для добавил по интуиции, тк кажется что надо больше сфокусироваться на предсказании конца и начала фразы
        # Конец предсказывать сложнее, поэтому вес больше 
        # Порядок: ['O', 'B-EXEC', 'B-GUAR', 'I-EXEC', 'I-GUAR', 'E-EXEC', 'E-GUAR']
        self.criterion = nn.CrossEntropyLoss(weight=torch.tensor([1/20, 1, 1, 1/4, 1/4, 4, 4]))

    def freeze_bert_layers(self):
        # Заморозим все слои, кроме тех, которые указаты в кофиге
        for name, param in self.bert.named_parameters():
            if name.startswith(tuple(CONFIG.model.unfreeze_layers)):
                param.requires_grad = True
            else:
                param.requires_grad = False

        # линейные слои тоже оставим в тепле
        for param in self.linear.parameters():
            param.requires_grad = True

    def forward(self, input_ids, token_type_ids, attention_mask):
        bert_output = self.bert(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)
        sequence_output = bert_output.last_hidden_state

        return self.linear(sequence_output).squeeze(-1)

    def training_step(self, batch, batch_idx):
        input_ids, token_type_ids, attention_mask, token_tags = batch

        # зарешейпим тензоры в подходящий для torch.CE размер
        logits = self.forward(input_ids, token_type_ids, attention_mask).reshape(-1, len(tags)) # (batch_size, 512, 7) -> (batch_size * 512, 7)
        token_tags = token_tags.reshape(-1) # (batch_size, 512) -> (batch_size * 512)

        loss = self.criterion(logits, token_tags)

        self.log("train/loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        input_ids, token_type_ids, attention_mask, token_tags = batch
        logits = self.forward(input_ids, token_type_ids, attention_mask).reshape(-1, len(tags))
        token_tags = token_tags.reshape(-1)
        
        loss = self.criterion(logits, token_tags)

        self.log("val/loss", loss)
        return loss
    
    def test_step(self, batch, batch_idx):
        tp = 0
        input_ids, token_type_ids, attention_mask, token_tags = batch
        # сделаем предикт
        logits = self.forward(input_ids, token_type_ids=token_type_ids, attention_mask=attention_mask)

        # бежим по батчу
        for i in range(len(batch)):
            if tags.index('B-EXEC') in token_tags[i] and tags.index('E-EXEC') in token_tags[i]:
                start_num = tags.index('B-EXEC')
                end_num = tags.index('E-EXEC')
            elif tags.index('B-GUAR') in token_tags[i] and tags.index('E-GUAR') in token_tags[i]:
                start_num = tags.index('B-GUAR')
                end_num = tags.index('E-GUAR')
            else:
                # тут по хорошему надо протестить с обоими тэгами, но я не успел
                start_num = tags.index('B-EXEC')
                end_num = tags.index('E-EXEC')

            # достанем самые вероятные начало и конец
            pred_start = torch.argmax(logits[i, :, start_num]).item()
            pred_end = torch.argmax(logits[i, :, end_num]).item()
            
            # если у одного из токенов самый вероятный класс это не начало или конец соответственно
            # то забаним все предложение
            if torch.argmax(logits[i, pred_start]) != start_num:
                pred_start, pred_end = 0, 0
            if torch.argmax(logits[i, pred_end]) != end_num:
                pred_start, pred_end = 0, 0

            try: # если проблема с индексами, то значит в этом сэмпле пусто
                # тк одно слово может распасться на несколько токенов для start возьмем номер первого токена [0]
                real_start = torch.where(token_tags[i] == start_num)[0][0].item()         
                # а для end номер последнего токена [-1]
                real_end = torch.where(token_tags[i] == end_num)[0][-1].item()
            except IndexError:
                real_start, real_end = 0, 0

            if pred_start == real_start and pred_end == real_end:
                tp += 1

        return {
            "acc": tp,
            "batch_len": len(batch), # также вернем количество сэпилов на которых проверяли
        }

    # посчитаем итоговый accuracy
    def test_epoch_end(self, outputs):
        metrics = {
            "acc": 0,
        }
        test_loader_len = 0
        for batch in outputs:
            test_loader_len += batch["batch_len"]
            for key in metrics:
                metrics[key] += batch[key]

        for key in metrics:
            metrics[key] /= test_loader_len

        print(metrics)
        return metrics

    def configure_optimizers(self):
        optimizer = Adam(self.parameters(), lr=self.lr) # в качестве оптимизоватора возьмем просто Adam. тк работает
        scheduler = torch.optim.lr_scheduler.LinearLR( # со scheduler'ами тоже можно было бы поэкспериментировать
            optimizer=optimizer,
            start_factor=1,
            end_factor=0.00001,
            total_iters=10_000,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
        }

## Обучение

In [26]:
data_module = TextExtractionDataModule(batch_size=CONFIG.training.batch_size)

Скачаем веса модельки, которую я обучил  
Веса положил на гугл диск

In [27]:
!bash download_weights.sh

File rubert-base-cased-ner-kontur.ckpt already exists in the weights/ directory. No need to download.


Тут можно выбрать, взять уже обученную на всех данных мною модель  
или претрейн от DeepPavlov, с которого я тренировал модель

In [28]:
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=['Не выбрано', 'Обученная модель', 'Претрейн от DeepPavlov'],
    value='Не выбрано',
    description='Выберите веса',
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        global model
        if change['new'] == 'Обученная модель':
            path = "weights/rubert-base-cased-ner-kontur.ckpt"
            model = BertForTextExtraction.load_from_checkpoint(path, model_name=CONFIG.model.model_name)
            print(f"Взяты веса из {path}")
        elif change['new'] == 'Претрейн от DeepPavlov':
            model = BertForTextExtraction(CONFIG.model.model_name)
            print("Взяты претрейн веса")
        else:
            print("Выберете еще раз")

dropdown.observe(on_change)

display(dropdown)

Dropdown(description='Выберите веса', options=('Не выбрано', 'Обученная модель', 'Претрейн от DeepPavlov'), va…

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Взяты претрейн веса


Создадим Trainer. Он будет сохранять лучший чекпоит по train_loss и отдельно по val_loss, а также просто последний чекпоинт  
Метрики будут логироваться в wandb (для этого надо авторизироваться командой ниже, или закомментить логгер)

In [29]:
# !wandb login <your_token>

In [ ]:
trainer = pl.Trainer(
    default_root_dir='./checkpoints/lr/',
    log_every_n_steps=10,
    precision=CONFIG.training.precision,
    auto_lr_find=True,
    accelerator='gpu',
    devices=1,
    max_epochs=CONFIG.training.epochs,
    logger=pl.loggers.WandbLogger( # это можно закомментить
        project=CONFIG.training.project_name,
        name=CONFIG.training.experiment_name,
        config=CONFIG.to_dict(),
    ),
    callbacks=[
        pl.callbacks.ModelCheckpoint(
            dirpath=f'checkpoints/{CONFIG.training.experiment_name}', filename='epoch={epoch}-val_loss={val/loss:.2f}',
            monitor='val/loss', save_top_k=1, mode='min', auto_insert_metric_name=False),
        pl.callbacks.ModelCheckpoint(
            dirpath=f'checkpoints/{CONFIG.training.experiment_name}', filename='epoch={epoch}-train_loss={train/loss:.2f}',
            monitor='train/loss', save_top_k=1, mode='min', auto_insert_metric_name=False),
        pl.callbacks.ModelCheckpoint(
            dirpath=f'checkpoints/{CONFIG.training.experiment_name}', filename='epoch={epoch}-train_loss={train/loss:.2f}',
            every_n_epochs=10, auto_insert_metric_name=False),
        pl.callbacks.LearningRateMonitor(logging_interval='epoch')
    ],
)

Подберем lr

In [31]:
trainer.tune(model, datamodule=data_module)

/home/gk/envs/dsenv/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:613: UserWarning: Checkpoint directory /home/gk/projects/kontur/solution/checkpoints/exp6-final-test exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

/home/gk/envs/dsenv/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:138: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
`Trainer.fit` stopped: `max_steps=100` reached.
Learning rate set to 4.786300923226385e-05
Restoring states from the checkpoint path at checkpoints/lr/.lr_find_4be8a57f-1ab0-48a4-b5bb-73974bf80240.ckpt
Restored all states from the checkpoint file at checkpoints/lr/.lr_find_4be8a57f-1ab0-48a4-b5bb-73974bf80240.ckpt


{'lr_find': <pytorch_lightning.tuner.lr_finder._LRFinder at 0x7f8780d7fca0>}

### Запустим обучение

In [32]:
trainer.fit(model, datamodule=data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | bert      | BertModel        | 177 M 
1 | linear    | Linear           | 5.4 K 
2 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
14.2 M    Trainable params
163 M     Non-trainable params
177 M     Total params
355.718   Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=30` reached.


## Тестирование

Оценим точность на не пустых текстах из test dataloader'а

In [38]:
trainer.checkpoint_callback.best_model_path

'/home/gk/projects/kontur/solution/checkpoints/exp6-final-test/epoch=7-val_loss=0.09.ckpt'

In [41]:
model = BertForTextExtraction.load_from_checkpoint(
    trainer.checkpoint_callback.best_model_path,
    model_name=CONFIG.model.model_name
)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [42]:
trainer.test(model, datamodule=data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

{'acc': 0.71875}


[{}]

In [44]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.eval().to(device)

### Оценим точность на final_test_data, которую мы отложили в начале

Для предсказания фразы на длинном тексте используется скользящее окно в 400 токенов, с шагом 100. Тоесть соседние окна будут пересекаться на 300 токенов.  
Это сделанно для того, чтобы хотя бы в одном окне фраза попала от начала до конца (как в обучающей выборке)  
За итоговый ответ я беру предсказанное начало и конец из того окна где уверенность токена начала + уверенность токена конца максимальна.  
Также из если предсказанная фраза оканчивается на что-то из ",/:" я удаляю этот символ, тк у модели не всегда получается их убирать

In [45]:
def inference(inference_data):
    predictions = []
    for item in tqdm(inference_data):
        text = item['text']
        label = item['label']

        if label == 'обеспечение исполнения контракта':
            short_label = 'EXEC'
            start_num = 1
            end_num = 5
        else:
            short_label = 'GUAR'
            start_num = 2
            end_num = 6

        encoding = tokenizer(
            text,
            add_special_tokens=True,
            return_attention_mask=True,
            return_token_type_ids=True,
            return_offsets_mapping=True,
            return_tensors='pt'
        )

        sentence_len = len(encoding['attention_mask'][0])
        step = 100
        window_size = 400
        final_seq_len = 512

        potential_answers = []

        offset = 0
        for i in range(0, sentence_len, step):
            offset = i
            data = (
                pad_to_length(torch.cat((
                    labels_tokenized[short_label]["data"]['input_ids'][0],
                    encoding['input_ids'][0][i:i+window_size],
                )), final_seq_len, 0).to(device).unsqueeze(0),
                pad_to_length(torch.cat((
                    labels_tokenized[short_label]["data"]['token_type_ids'][0],
                    encoding['token_type_ids'][0][i:i+window_size],
                )), final_seq_len, 0).to(device).unsqueeze(0),
                pad_to_length(torch.cat((
                    labels_tokenized[short_label]["data"]['attention_mask'][0],
                    encoding['attention_mask'][0][i:i+window_size],
                )), final_seq_len, 0).to(device).unsqueeze(0),
            )
            logits = model(*data)[0]
            
            # найдем токен у которого самая большая вероятность что он начало
            pred_start_idx = torch.argmax(logits[:, start_num])
            # если у этого токена самый вероятный класс это не начало, то пропустим все окно
            # тк будем считать что в нем не находится начало
            if torch.argmax(logits[pred_start_idx]) != start_num:
                continue
            pred_start_conf = logits[pred_start_idx, start_num]

            # токен конца будем уже выбирать из токенов, которые идут за токеном начала (тк начало легче предсказывать)
            pred_end_idx = torch.argmax(logits[pred_start_idx:, end_num]) + pred_start_idx
            if torch.argmax(logits[pred_end_idx]) != end_num:
                continue
            pred_end_conf = logits[pred_end_idx, end_num]

            # к индексам найденых токенов добавим сдиг окна
            # и вычтем длинну запроса, который мы добавляли в начале каждого окна
            pred_start_idx += offset - labels_tokenized[short_label]["len"]
            pred_end_idx += offset - labels_tokenized[short_label]["len"]
            # добавим найденную пару в потенциальные ответы
            potential_answers.append((
                pred_start_idx,
                pred_end_idx,
                pred_start_conf + pred_end_conf # в качестве уверенности ответа возьмем сумму уверенностей
            ))

        # достанем пару с самой большой уверенностью
        if potential_answers:
            best_borders = max(potential_answers, key=lambda x: x[2])
        else:
            # если ни одна паре на нашлась, то будут нули
            best_borders = (0, 0)

        # если тут вывалилась ошибка, то модель предсказала токен [PAD]
        try:
            # достанем индекс первого символа в токене начала
            start_char_idx = encoding['offset_mapping'][0][best_borders[0]][0].item()
            end_char_idx = encoding['offset_mapping'][0][best_borders[1]][1].item()
        except:
            # предположим, что ответа в тексте нет
            start_char_idx, end_char_idx = 0, 0
            # можно доработать и пребрать другие предположения из potential_answers TODO

        pred = {
            "text": [text[start_char_idx:end_char_idx]],
            "answer_start": [start_char_idx],
            "answer_end": [end_char_idx]
        }
        
        if pred["text"][0] != "":
            # удалим лишние знаки
            if pred["text"][0][-1] in ",/:;-_":
                pred["text"][0] = pred["text"][0][:-1]
                pred["answer_end"][0] -= 1
                
            if pred["text"][0][0] in ["."]:
                pred["text"][0] = pred["text"][0][1:]
                pred["answer_start"][0] += 1

        predictions.append(pred)
    return predictions

In [46]:
data = final_test_data
predictions = inference(data)
tp = 0
diff = []
for gt, pred in zip(data, predictions):
    if gt['extracted_part'] == pred:
        tp += 1
    else:
        diff.append([gt["id"], pred, gt['extracted_part']])


print(tp / len(data))

with open("diff.json", 'w', encoding='utf-8') as f:
    json.dump(diff, f, indent=2, ensure_ascii=False)

  0%|          | 0/180 [00:00<?, ?it/s]

0.8222222222222222


## Сгенерируем submission

In [46]:
with open(CONFIG.data.test_data_path, encoding='utf-8') as f:
    data = json.load(f)

submission = []
predictions = inference(data)
for item, pred in zip(data, predictions):
    item['extracted_part'] = pred
    submission.append(item)

  0%|          | 0/318 [00:00<?, ?it/s]

In [47]:
with open("predictions.json", 'w', encoding='utf-8') as f:
    json.dump(submission, f, indent=2, ensure_ascii=False)